## Environment Setting
Google drive mount (for Colab users) and package importing.
You can optionally install and import torchensemble package for ensemble learning

In [ ]:
import os
import numpy as np
import torch
import torch.nn as nn
import random  
import torchvision.transforms as transforms
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import DataLoader

## (Optional) Sample Visualization
You can see actual sample images and sorted class indices. Additional matplotlib package is needed.

In [ ]:
# Just for reference: see actual samples
import matplotlib.pyplot as plt

alphabet = {
        'A(a)' : '0', 'B(b)' : '1', 'C(c)' : '2', 'D(d)' : '3', 'E(e)' : '4', 'F(f)' : '5', 
        'G(g)' : '6', 'H(h)' : '7', 'I(i)' : '8', 'J(j)' : '9', 'K(k)' : '10','L(l)' : '11', 
        'M(m)' : '12', 'N(n)' : '13', 'O(o)' : '14', 'P(p)' : '15', 'Q(q)' : '16', 'R(r)' : '17', 
        'S(s)' : '18', 'T(t)' : '19', 'U(u)' : '20', 'V(v)' : '21', 'W(w)' : '22', 'X(x)' : '23', 
        'Y(y)' : '24', 'Z(z)' : '25'
    }

In [ ]:
# Just for reference: see actual samples

load_sample = np.load('./sample_data.npy', allow_pickle=True).item()
# load_sample = np.load('/content/drive/MyDrive/final_proj_colab/emnist_progress_easy_data/sample_data.npy', allow_pickle=True).item()
sample_data, sample_label = load_sample['train_data'], load_sample['train_label']
print(len(sample_data))


plt.figure(figsize=(len(sample_data),len(sample_data)))
for i in range(len(sample_data)):
    plt.subplot(1, len(sample_data), i+1)
    ax = plt.gca()
    ax.axes.xaxis.set_ticklabels([])
    ax.axes.yaxis.set_ticklabels([])
    plt.imshow(sample_data[i], cmap='gray')
    
plt.show()
print("progress label: ", end=' ')
label_str = '('

for i in range(len(sample_label)):
    print(int(sample_label[i]), end=' ')
    label_str += " " + list(alphabet.keys())[int(sample_label[i])]
label_str += " )"
print()
print(label_str)

In [ ]:
# Use 0th GPU for training
torch.cuda.set_device(0)

In [ ]:
# fix random seed to increase reproducibility
# NOTE: Do not modify here!

random_seed = 7
torch.manual_seed(random_seed)
os.environ['PYTHONHASHSEED'] = str(random_seed)
np.random.seed(random_seed)
random.seed(random_seed)
torch.cuda.manual_seed(random_seed)

torch.backends.cudnn.benchmark = False
torch.backends.cudnn.deterministic = True
%env CUBLAS_WORKSPACE_CONFIG=:16:8

def seed_worker(worker_seed):
    np.random.seed(worker_seed)
    random.seed(worker_seed)
    
# you can modify this
num_workers = 1

In [ ]:
# NOTE: you can modify mean and std for normalization
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.1307,), (0.3081,)),
])

batch_size = 256

In [ ]:
# NOTE: modify path for your setting

from data_utils import Mydataset, collate_fn

train_path = './train'
valid_path = './valid'
# train_path = '/content/drive/MyDrive/final_proj_colab/emnist_progress_easy_data/train'
# valid_path = '/content/drive/MyDrive/final_proj_colab/emnist_progress_easy_data/valid'

train_ds = Mydataset(train_path, transform=transform, train=True)
valid_ds = Mydataset(valid_path, transform=transform, train=False)

train_dl = DataLoader(train_ds, batch_size=batch_size, collate_fn=collate_fn, shuffle=True)
valid_dl= DataLoader(valid_ds, batch_size=batch_size, collate_fn=collate_fn, shuffle=False)

In [ ]:
import time

def train(model, model_optim, loss_func, max_epoch, train_dl, valid_dl, 
          load_path=None, save_path='./model.pt'):
    ##############################################################################
    #                          IMPLEMENT YOUR CODE                               #
    ##############################################################################
    # Load your states
    loaded_epoch = 0
    loaded_best_acc = -1
    if load_path is not None:
        state = torch.load(load_path)
        model.load_state_dict(state["model"])
        model_optim.load_state_dict(state["optimizer"])
        loaded_epoch = state["epoch"]
        loaded_best_acc = state["best_acc"]
        # ...
    start_time = time.time()
        
    ##############################################################################
    #                          END OF YOUR CODE                                  #
    ##############################################################################
    
    best_valid_accuracy = 0 if loaded_best_acc == -1 else loaded_best_acc

    for epoch in np.array(list(range(max_epoch - loaded_epoch))) + loaded_epoch:
        n_samples = 0
        n_correct = 0
        model.train()
        for step, sample in enumerate(train_dl):
            img, label = sample

            model_optim.zero_grad()
            outputs = model((img, label))
            
            ##############################################################################
            #                          IMPLEMENT YOUR CODE                               #
            ##############################################################################
            # Problem 4: implement optimization part

            loss = loss_func(outputs, label)
            loss.backward()
            model_optim.step()
            
            ##############################################################################
            #                          END OF YOUR CODE                                  #
            ##############################################################################
            
            # you can modify below train evaluation code
            n_samples += len(label)
            for j in range(len(label)):
                n_correct += (outputs[j].argmax(-1)[-1] == label[j][-1].cuda()).sum().item()
            
            if (step + 1) % print_interval == 0:
                print('epoch:', epoch + 1, 'step:', step + 1, 'loss:', loss.item(), 'accuracy:', 100 * (n_correct / n_samples))
                elapsed_time = time.time() - start_time
                print('elapsed time : %d h %d m %d s' % (elapsed_time / 3600, (elapsed_time % 3600) / 60, (elapsed_time % 60)))            
                
        # you can modify evaluation code
                
        n_samples = 0
        n_correct = 0
        with torch.no_grad():
            model.eval()
            for step, sample in enumerate(valid_dl):
                img, label = sample            
                outputs = model(img)
                n_samples += len(label)                
                for j in range(len(label)):
                    n_correct += (outputs[j].argmax(-1)[-1] == label[j][-1].cuda()).sum().item()
            
            valid_accuracy = 100 * (n_correct/n_samples)
            if valid_accuracy > best_valid_accuracy:
                print("New best valid accuracy, saving model")
                ##############################################################################
                #                          IMPLEMENT YOUR CODE                               #
                ##############################################################################
                # Save your states (optional)
                state = {
                    "model": model.state_dict(),
                    "optimizer": model_optim.state_dict(),
                    "epoch": epoch + 1,
                    "best_acc": best_valid_accuracy,
                    # ...
                }
                ##############################################################################
                #                          END OF YOUR CODE                                  #
                ##############################################################################
                torch.save(state, save_path)
                best_valid_accuracy = valid_accuracy
            print('Valid epoch: %d, Valid accuracy: %.2f, Best valid accuracy: %.2f' % (epoch + 1, valid_accuracy, best_valid_accuracy))

# you can modify evaluation code
def eval(valid_dl, load_path):
    state = torch.load(load_path)
    model.load_state_dict(state["model"])
    ##############################################################################
    #                          IMPLEMENT YOUR CODE                               #
    ##############################################################################
    # Problem 5: implement evaluation part
    # you can simply copy or modify above evaluation code in train function
    n_samples = 0
    n_correct = 0
    with torch.no_grad():
        model.eval()
        for step, sample in enumerate(valid_dl):
            img, label = sample
            outputs = model(img)
            n_samples += len(label)
            for j in range(len(label)):
                n_correct += (outputs[j].argmax(-1)[-1] == label[j][-1].cuda()).sum().item()

    ##############################################################################
    #                          END OF YOUR CODE                                  #
    ##############################################################################
        
    valid_accuracy = 100 * (n_correct/n_samples)
    print('Valid accuracy: %.2f' % (valid_accuracy))

In [ ]:
# You can add or modify your ConvLSTM's hyperparameter (keys and values)
kwargs = {
    'cnn_input_dim': 1,
    'cnn_hidden_size': 256,
    'rnn_input_dim': 256,
    'rnn_hidden_size': 32,
    'rnn_num_layers': 2,
    'rnn_dropout': 1
}

NUM_CLASSES = 26
SEQUENCE_LENGTH = 5

In [ ]:
# for reload .py file without restart
import models_easy
import importlib

importlib.reload(models_easy)

from models_easy import ConvLSTM

model = ConvLSTM(sequence_length=SEQUENCE_LENGTH, num_classes=NUM_CLASSES, **kwargs).cuda()
print(model)
##############################################################################
#                          IMPLEMENT YOUR CODE                               #
##############################################################################
model_optim = optim.Adam(params=model.parameters(), lr=1e-4)
loss_func = nn.CrossEntropyLoss()
##############################################################################
#                          END OF YOUR CODE                                  #
##############################################################################

In [ ]:
# NOTE: you can modify hyperparameters

print_interval = 15
max_epoch = 1

In [ ]:
load_path = None
train(model, model_optim, loss_func, max_epoch, train_dl, valid_dl, load_path=load_path, save_path='./model.pt')

In [ ]:
# load and evaluate model
load_path = './model.pt'
eval(valid_dl, load_path)

### Test code for grading by TA

In [ ]:
# you do not need to modify here
from data_utils import Mydataset, collate_fn

test_path = './data/emnist_progress_easy_data/test'
test_ds = Mydataset(test_path, transform=transform, train=False)
test_dl= DataLoader(test_ds, batch_size=batch_size, collate_fn=collate_fn, shuffle=False)

In [ ]:
# please change the model name to your submission model name
load_path = './model.pt'
eval(test_dl, load_path)